In [8]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
#import yfinance as yf
import os

In [3]:
# Función para obtener datos financieros
def get_financial_data(tickers_dict, start_date=None):
    """
    Obtiene datos financieros históricos para diferentes tipos de instrumentos.
    
    Parameters:
    tickers_dict (dict): Diccionario con categorías e instrumentos
    start_date (str): Fecha de inicio para datos históricos
    
    Returns:
    dict: Diccionario con DataFrames por categoría
    """
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=3*365)).strftime('%Y-%m-%d')
    
    data_dict = {}
    
    for category, tickers in tickers_dict.items():
        category_data = {}
        for ticker in tickers:
            try:
                # Obtener datos históricos
                stock = yf.Ticker(ticker)
                hist = stock.history(start=start_date)
                
                # Obtener información adicional
                info = stock.info
                
                # Calcular métricas adicionales
                if len(hist) > 0:
                    hist['Daily_Return'] = hist['Close'].pct_change()
                    hist['Volatility'] = hist['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
                    
                    # Agregar información básica
                    category_data[ticker] = {
                        'historical_data': hist,
                        'current_price': hist['Close'].iloc[-1] if len(hist) > 0 else None,
                        'market_cap': info.get('marketCap', None),
                        'beta': info.get('beta', None),
                        'sector': info.get('sector', None)
                    }
            
            except Exception as e:
                print(f"Error obteniendo datos para {ticker}: {str(e)}")
                continue
        
        data_dict[category] = category_data
    
    return data_dict

In [4]:
# Definir los instrumentos financieros (5 por categoría)
financial_instruments = {
    'CEDEARS': [
        'AAPL.BA',  # Apple
        'GOOGL.BA', # Google
        'AMZN.BA',  # Amazon
        'TSLA.BA',  # Tesla
        'MELI.BA'   # MercadoLibre
    ],
    'ACCIONES_ARG': [
        'GGAL.BA',  # Grupo Financiero Galicia
        'YPF.BA',   # YPF
        'PAMP.BA',  # Pampa Energía
        'BBAR.BA',  # Banco BBVA Argentina
        'TXAR.BA'   # Ternium Argentina
    ],
    'ETFs': [
        'SPY',      # S&P 500 ETF
        'QQQ',      # Nasdaq ETF
        'EEM',      # Emerging Markets ETF
        'GLD',      # Gold ETF
        'SLV'       # Silver ETF
    ]
}

In [5]:
# Función para calcular métricas del portafolio
def calculate_portfolio_metrics(data_dict):
    """
    Calcula métricas importantes para cada instrumento y categoría.
    
    Parameters:
    data_dict (dict): Diccionario con datos financieros
    
    Returns:
    pd.DataFrame: DataFrame con métricas por instrumento
    """
    metrics_list = []
    
    for category, instruments in data_dict.items():
        for ticker, data in instruments.items():
            hist_data = data['historical_data']
            
            if len(hist_data) > 0:
                # Calcular métricas
                returns = hist_data['Daily_Return'].dropna()
                
                metrics = {
                    'Category': category,
                    'Instrument': ticker,
                    'Current_Price': data['current_price'],
                    'Annual_Return': returns.mean() * 252 * 100,  # Anualizado en porcentaje
                    'Annual_Volatility': returns.std() * np.sqrt(252) * 100,  # Anualizada en porcentaje
                    'Sharpe_Ratio': (returns.mean() * 252) / (returns.std() * np.sqrt(252)) if returns.std() != 0 else 0,
                    'Max_Drawdown': ((hist_data['Close'].cummax() - hist_data['Close']) / hist_data['Close'].cummax()).max() * 100,
                    'Market_Cap': data['market_cap'],
                    'Beta': data['beta'],
                    'Sector': data['sector']
                }
                
                metrics_list.append(metrics)
    
    return pd.DataFrame(metrics_list)

In [6]:
# Función para exportar datos
def export_data(data_dict, metrics_df, export_path='financial_data/'):
    """
    Exporta los datos y métricas a archivos CSV.
    
    Parameters:
    data_dict (dict): Diccionario con datos financieros
    metrics_df (pd.DataFrame): DataFrame con métricas
    export_path (str): Ruta para exportar los archivos
    """
    # Crear directorio si no existe
    if not os.path.exists(export_path):
        os.makedirs(export_path)
    
    # Exportar métricas del portafolio
    metrics_df.to_csv(f'{export_path}portfolio_metrics.csv', index=False)
    
    # Exportar datos históricos por instrumento
    for category, instruments in data_dict.items():
        for ticker, data in instruments.items():
            data['historical_data'].to_csv(f'{export_path}{ticker}_historical.csv')

In [7]:
# Función principal para ejecutar todo el proceso
def main():
    # Obtener datos
    print("Obteniendo datos financieros...")
    data_dict = get_financial_data(financial_instruments)
    
    # Calcular métricas
    print("Calculando métricas...")
    metrics_df = calculate_portfolio_metrics(data_dict)
    
    # Exportar datos
    print("Exportando datos...")
    export_data(data_dict, metrics_df)
    
    # Mostrar métricas
    print("\nMétricas del Portafolio:")
    print(metrics_df)
    
    return data_dict, metrics_df

In [ ]:
if __name__ == "__main__":
    data_dict, metrics_df = main()